<a href="https://colab.research.google.com/github/manjujoby/regression/blob/main/LSTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv('Education.csv')

In [2]:
df.head()


,Text,Label
0,The impact of educational reforms remains unce...,positive
1,Critics argue that recent improvements in the ...,negative
2,Innovative teaching methods have led to unexpe...,positive
3,"Despite budget constraints, the school has man...",positive
4,The true effectiveness of online learning plat...,negative


In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Label']=le.fit_transform(df['Label'])

In [5]:
df['Label'].head()

,Label
0,1
1,0
2,1
3,1
4,0


In [6]:
y=df['Label']

In [8]:
x=df['Text']

In [9]:
x.head()

,Text
0,The impact of educational reforms remains unce...
1,Critics argue that recent improvements in the ...
2,Innovative teaching methods have led to unexpe...
3,"Despite budget constraints, the school has man..."
4,The true effectiveness of online learning plat...


In [12]:
from tensorflow.keras.preprocessing.text import one_hot

In [14]:
n=df.shape[0]
n

52

In [15]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
corpus=[]
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
for i in range(0,n):
  review=re.sub('[^a-zA-Z]',' ',x[i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word)for word in review if word not in set(stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)



In [18]:
corpus[0]

'impact educ reform remain uncertain despit extens research'

In [20]:
from tensorflow.keras.preprocessing.text import one_hot

In [23]:
voc_size=1000
onehotted=[one_hot(word,voc_size)for word in corpus]

In [25]:
onehotted[0]

[168, 246, 873, 194, 11, 352, 216, 451]

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sen_len=15
padded=pad_sequences(onehotted,maxlen=sen_len,padding='pre')

In [28]:
padded[0]

array([  0,   0,   0,   0,   0,   0,   0, 168, 246, 873, 194,  11, 352,
       216, 451], dtype=int32)

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Bidirectional


In [47]:
feature_dim=30
model=Sequential()
model.add(Embedding(voc_size,feature_dim,input_length=15))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [60]:
y=df['Label']

In [61]:
import numpy as np
x=np.array(padded)
y=np.array(y)

In [62]:
y

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0])

In [63]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [65]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=30,batch_size=5)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6564 - loss: 0.5889 - val_accuracy: 0.3636 - val_loss: 0.7070
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9532 - loss: 0.3443 - val_accuracy: 0.5455 - val_loss: 1.0914
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9599 - loss: 0.0808 - val_accuracy: 0.5455 - val_loss: 2.0554
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9824 - loss: 0.0449 - val_accuracy: 0.6364 - val_loss: 1.4640
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.0090 - val_accuracy: 0.5455 - val_loss: 1.7969
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9898 - loss: 0.0144 - val_accuracy: 0.5455 - val_loss: 1.9537
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.5455 - val_loss: 1.8513
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.5455 - val_loss: 1.8679


In [67]:
y_pred=model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [70]:
from sklearn.metrics import accuracy_score,confusion_matrix
a=accuracy_score(y_test,y_pred.round())

In [71]:
a

0.5454545454545454

In [73]:
c=confusion_matrix(y_test,y_pred.round())
c

array([[2, 2],
       [3, 4]])